In [10]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import fits

pathlist = glob.glob('/blue/adamginsburg/richardson.t/research/flux/robitaille_models-1.1/s*')

geometry = [os.path.basename(path).split('/')[-1][:7] for path in pathlist]

b3fluxarr = b6fluxarr = massarr = []
for path in geometry:
    filepath = './'+path+'/flux_prop.fits'
    if not os.path.isfile(filepath):
        continue
    fluxfile = fits.open(filepath)
    cols = fluxfile[1].columns
    if not 'mass' in cols.names:
        continue
    

    b6flux = fluxfile[1].data['B6 flux'][:,3]
    b3flux = fluxfile[1].data['B3 flux'][:,3]
            
    dusttemp = fluxfile[1].data['mass'][:,3]
    notemp = dusttemp>-99
    dusttemp = dusttemp[notemp]
    logtemp = np.log10(dusttemp)

    b3flux = b3flux[notemp] ; b6flux = b6flux[notemp]

    b3fluxarr.append(list(b3flux)) ; b6fluxarr.append(list(b6flux))
    massarr.append(list(logtemp))

from sklearn.model_selection import train_test_split

X = np.vstack((b3fluxarr, b6fluxarr)).T
y = np.array(massarr)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

from sklearn.preprocessing import StandardScaler

X_train_scaled = StandardScaler().fit(X_train).transform(X_train)
X_test_scaled = StandardScaler().fit(X_train).transform(X_test)



(77247,)
(9149,)
(78291,)
(322335,)


/scratch/local/60751837/ipykernel_6819/1818973568.py:28: RuntimeWarning: divide by zero encountered in log10
  logtemp = np.log10(dusttemp)


(81198,)
(70218,)
(77688,)
(73098,)
(74304,)
(75942,)
(8559,)
(0,)
(659349,)
(0,)
(8481,)
(1287,)
(48, 2)
(48,)


/home/t.yoo/anaconda3/lib/python3.9/site-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)
/scratch/local/60751837/ipykernel_6819/1818973568.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = np.array(massarr)


ValueError: setting an array element with a sequence.

In [ ]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(100,100),activation="relu",
                      max_iter=1000,random_state=1,verbose=1)

model.fit(X_train_scaled, y_train)

y_pred=model.predict(X_test_scaled)